Scripts compiled here are scripted in python on Rhino-GH for use in:
1. Base Grid - Topology Extraction & Export to JSON
2. Allocation of Modules in Rhino - Read JSON output
3. Sort Module types to Orientation Logic in Rhino - Sort Module to Logic

In [ ]:
"""     1. Base Grid - Topology Extraction & Export to JSON     """

"""Provides a scripting component.
    Inputs:
        nodes_index: List of points, flattened
        ms_nodes_list: Index of Medium Street Nodes
        ls_nodes_index: Index of Local Street Nodes
        x_nodes_index: Index of External Nodes
        ls_avail_nodes_index: Index of Local Street available nodes
        ms_avail_nodes_index: Index of Medium Street available nodes
        edge_list: Tree of Edges with each branch representing a node
        layer_nodes_index: Tree with each branch representing a "step" from the street, and items in branch as index of nodes on that "step"
        point_count: Int of number of points in 1 level of the site
        stair_nodes: List of Layer 1 nodes which represents a staircase module
        fpath_nodes: Filepath for Nodes file
        fpath_edges: Filepath for Edges file
        run: False = No writing to file, True = Write
"""

__author__ = "junwenloo"

import rhinoscriptsyntax as rs
from ghpythonlib import treehelpers as th
import json

#obtained definition from stack overflow, https://stackoverflow.com/questions/38987/how-do-i-merge-two-dictionaries-in-a-single-expression-taking-union-of-dictiona
def merge_two_dicts(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

"""
    Node Attributes graph creation, filled only with 'tag' and 'use_frequency' data
Key as nodes and values of each nodes as a dictionary of attributes:
"name" - Node index
'district' - Based on user's district
"type" - None
"use_frequency - "D" (Local Street, Daily), "M" (Medium Street, Monthly) [[Future work - "O" (Main Street, Occasional)]]
"units" - None 
"tag" - None (available node), 0 (Local Street), 1 (Medium Street), 2 (Main Street), 3 (Staircase), 4 (2nd Level Street), "x" (External Node)
"layer" - 1,2,3,4,5 etc - Represents the number of steps from the street
"""

nodes_1 = {}
nodes_2 = {}


layer_nodes_index = th.tree_to_list(layer_nodes_index, None)

for i in nodes_index:
    
    half = int((len(nodes_index))/2)
    
    if i < half:
        #level 1 nodes

        n_attr_dict = {}
        n_attr_dict["name"] = i
        n_attr_dict['district'] = 7
        n_attr_dict['type'] = None
        n_attr_dict['units'] = None

        if i in ms_nodes_index:
            #medium street attributes
            n_attr_dict['use_frequency'] = "M"
            n_attr_dict['tag'] = 1
            
        elif i in ls_nodes_index:
            #local street attributes
            n_attr_dict['use_frequency'] = "D"
            n_attr_dict['tag'] = 0
        
        elif i in x_nodes_index:
            #external nodes attributes
            n_attr_dict['use_frequency'] = None
            n_attr_dict['tag'] = "x"

        elif i in ls_avail_nodes_index:
            #local street available nodes
            n_attr_dict['use_frequency'] = "D"
            
            #to locate the stair nodes
            if i in stair_nodes:
                n_attr_dict['tag'] = 3
            else:
                n_attr_dict['tag'] = None
                
        elif i in ms_avail_nodes_index:
            #medium street available nodes
            n_attr_dict['use_frequency'] = "M"
            
            #to locate the stair nodes
            if i in stair_nodes:
                n_attr_dict['tag'] = 3
            else:
                n_attr_dict['tag'] = None

        nodes_1[int(i)] = n_attr_dict
    
    else:
        #level 2 nodes
        
        n_attr_dict = {}
        n_attr_dict["name"] = i
        n_attr_dict['district'] = 7
        n_attr_dict['type'] = None
        n_attr_dict['units'] = None
            
        if i-point_count in ms_nodes_index:
            #medium street attributes
            n_attr_dict['use_frequency'] = "M"
            n_attr_dict['tag'] = 1
            
        elif i-point_count in ls_nodes_index:
            #local street attributes
            n_attr_dict['use_frequency'] = "D"
            n_attr_dict['tag'] = 0
        
        elif i-point_count in x_nodes_index:
            #external nodes attributes
            n_attr_dict['use_frequency'] = None
            n_attr_dict['tag'] = "x"

        elif i-point_count in ls_avail_nodes_index:
            #local street available nodes
            n_attr_dict['use_frequency'] = "D"
            
            #to locate the stair nodes
            if i in stair_nodes:
                n_attr_dict['tag'] = 3
            else:
                n_attr_dict['tag'] = None
        
        elif i-point_count in ms_avail_nodes_index:
            #medium street available nodes
            n_attr_dict['use_frequency'] = "M"
            
            #to locate the stair nodes
            if i in stair_nodes:
                n_attr_dict['tag'] = 3
            else:
                n_attr_dict['tag'] = None
        
        nodes_2[int(i)] = n_attr_dict


#adding layer information
for index, step in enumerate(layer_nodes_index):
    for node in step:
            #for level 1 nodes
            nodes_1[node]["layer"] = int(index + 1)
            #for 2nd level nodes
            nodes_2[node + point_count]["layer"] = int(index + 1)

"""
    Create Edge list as tuples of (Start_node, ened_node) to build graph in Networkx
"""

edge_list = th.tree_to_list(edge_list, None)

edges = []

for start_node in range(len(edge_list)):
    for end_node in edge_list[start_node]:
        
        edges.append((start_node, end_node))
        edges.append((start_node + point_count, end_node + point_count))
"""
    Optional: Combining Nodes_1 and Nodes_2 together into nodes.txt
"""

nodes = merge_two_dicts(nodes_1, nodes_2)

"""
    Writing JSON file of nodes.txt and edges.txt
"""

if run:
    
    with open(str(fpath_nodes), 'w') as outfile:
        json.dump(nodes, outfile)

    with open(str(fpath_nodes_1), 'w') as outfile:
        json.dump(nodes_1, outfile)

    with open(str(fpath_nodes_2), 'w') as outfile:
        json.dump(nodes_2, outfile)

    with open(str(fpath_edges), 'w') as outfile:
        json.dump(edges, outfile)



"""     2. Allocation of Modules in Rhino - Read JSON output    """

"""Provides a scripting component.
    Inputs:
        edges: Tree with Branch No. as nodes and items in branch as nodes it connects to
        file: Filepath to updated Nodes json file
    Output:
        connection: Types of connections (0 = Open, 1 =. Close)
        module: Module type required for each node
        all_edge: Tree with Branch no. = Index of node in node_list, and items in branch as nodes it connects to
        node_list: Nodes involved in allocation on site"""

__author__ = "junwenloo"

import rhinoscriptsyntax as rs
from ghpythonlib import treehelpers as th
import json

#reading exported Nodes JSON file
with open(str(file)) as json_file_nodes:
    nodes = json.load(json_file_nodes)
    nodes = {int(k):v for k, v in nodes.items()}
    
#reading exported Edges JSON file
with open(str(edges)) as json_file_edges:
    edges_list = json.load(json_file_edges)
    edges_list = {int(k):v for k, v in edges_list.items()}

#alternative to retrieve edges from previous steps, but this is inaccurate
"""edges = th.tree_to_list(edges, None)
edges_list = {}
for num, item in enumerate(edges):
    edges_list[num] = item"""

#edges_list = th.tree_to_list(edges, None)

node_list = []

all_edge_list = []
connection_list = []
module_list = []
names_list = []
names_dict = {}
type_list = []
counter = 0
j_count = []

for node in nodes:
    all_edge = edges_list[node]
    connection = nodes[node]["connection"]
    if nodes[node]["module"] != None:
        module = [nodes[node]["module"]]
        module_list.append(module)
        all_edge_list.append(all_edge)
        connection_list.append(connection)
        node_list.append(node)
        names_list.append(nodes[node]["tag"])
        type_list.append(nodes[node]["type"])
        
#count number of occurence for each name
for name in names_list:
    count = names_list.count(name)
    names_dict[name] = count

all_edge = th.list_to_tree(all_edge_list)
connection = th.list_to_tree(connection_list)
module = th.list_to_tree(module_list)



"""     3. Sort Module types to Orientation Logic in Rhino - Sort Module to Logic   """

"""Provides a scripting component.
    Inputs:
        module: Module Number combinations, ordered in sequence of nodes
    Output:
        index_list: List of index to apply to specific logic, each branch represent 1 logic
        module_index: Contains the index of the module types"""

__author__ = "junwenloo"

import rhinoscriptsyntax as rs
from ghpythonlib import treehelpers as th

#definition takes in a logic_list, the type of module to evaluate and the current index of the node and sorts it to the logic it
#belongs to and to a nested index_list to be distributed to the 8 different logics required to orient the modules
def append_index_type(logic_list, type, current_node):
    for n, logic in enumerate(logic_list):
        if type in logic:
            module_index.append(type_sequence.index(type))
            index_list[n].append(current_node)

module_index = []

#Each Logic represents a method of orientation, which applies to specific module types only
logic_1 = [10.1,10.2]
logic_2 = [20.1, 21.1, 20.2, 20.2]
logic_31 = [30.1, 32.1, 32.2, 30.2]
logic_32 = [31.1, 31.2]
logic_41 = [40.1, 42.1, 45.1, 45.2, 40.2, 42.2]
logic_42 = [43.1, 43.2]
logic_43 = [41.1, 41.2, 44.1, 44.2]
logic_51 = [50.1, 52.1, 50.2, 52.2]
logic_52 = [51.1, 51.2]


logic_list = [logic_1, logic_2, logic_31, logic_32,
                logic_41, logic_42, logic_43, logic_51, logic_52]

#sequence of types: Index position corresponds to its index in GH Merge component of all types, smallest to largest 
type_sequence = [10.1, 10.2, 20.1, 21.1, 20.1, 20.2, 30.1, 31.1, 32.1, 30.2, 31.2, 32.2, 40.1, 41.1, 42.1, 
                43.1, 44.1, 45.1, 40.2, 41.2, 42.2, 43.2, 44.2, 45.2, 50.1, 51.1, 52.1, 50.2, 51.2, 52.2]
                

#initiating the list of nodes which will require specifc orientation logic
logic_1_index = []
logic_2_index = []
logic_31_index = []
logic_32_index = []
logic_41_index = []
logic_42_index = []
logic_43_index = []
logic_51_index = []
logic_52_index = []

index_list = [logic_1_index, logic_2_index, logic_31_index, logic_32_index,
                logic_41_index, logic_42_index, logic_43_index, logic_51_index,
                logic_52_index]

#adding each nodes to the logic they apply to
for num, type in enumerate(module):
    append_index_type(logic_list, round(type, 1), num)

index_list = [logic_1_index, logic_2_index, logic_31_index, logic_32_index,
                logic_41_index, logic_42_index, logic_43_index, logic_51_index,
                logic_52_index]

for i in index_list:
    if len(i) == 0:
        i.append("Null")

index_list = th.list_to_tree(index_list, None)